In [42]:
import pandas as pd
import yfinance as yf

# 1. Load S&P 500 tickers from Wikipedia and fix symbols
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['symbol'] = sp500['Symbol'].str.replace('.', '-', regex=False)
symbols = sp500['symbol'].unique().tolist()

# 2. Set date range for data download
end_date = pd.Timestamp('2024-06-01')
start_date = end_date - pd.DateOffset(years=2)

# 3. Download historical data for all tickers with multi-level columns (attribute, ticker)
df = yf.download(
    tickers=symbols,
    start=start_date.strftime('%Y-%m-%d'),
    end=end_date.strftime('%Y-%m-%d'),
    group_by='ticker',
    auto_adjust=False,
    progress=True
)

# 4. Flatten multi-index columns: (attribute, ticker) -> ticker_attribute
df.columns = ['{}_{}'.format(ticker, attr) for attr, ticker in df.columns]

# 5. Convert wide to long format
df_long = df.reset_index().melt(id_vars='Date', var_name='ticker_attr', value_name='value')

# 6. Split 'ticker_attr' into 'ticker' and 'attribute'
df_long[['ticker', 'attribute']] = df_long['ticker_attr'].str.rsplit('_', n=1, expand=True)

# 7. Pivot to get columns: date, ticker, adj_close, close, high, low, open, volume
df_tidy = df_long.pivot_table(index=['Date', 'ticker'], columns='attribute', values='value').reset_index()

# 8. Rename columns to lowercase and consistent names
df_tidy = df_tidy.rename(columns={
    'Date': 'date',
    'Adj Close': 'adj_close',
    'Close': 'close',
    'High': 'high',
    'Low': 'low',
    'Open': 'open',
    'Volume': 'volume'
})

# 9. Optional: sort by date and ticker
df_tidy = df_tidy.sort_values(['date', 'ticker']).reset_index(drop=True)

# 10. Show sample output
print(df_tidy.head())


[*********************100%***********************]  503 of 503 completed

1 Failed download:
['SW']: YFPricesMissingError('possibly delisted; no price data found  (1d 2022-06-01 -> 2024-06-01) (Yahoo error = "Data doesn\'t exist for startDate = 1654056000, endDate = 1717214400")')


attribute       date     ticker           A        AAPL        ABBV  \
0         2022-06-01  Adj Close  120.500023  146.361862  130.349350   
1         2022-06-01      Close  123.089996  148.710007  146.020004   
2         2022-06-01       High  128.979996  151.740005  147.889999   
3         2022-06-01        Low  122.430000  147.679993  144.320007   
4         2022-06-01       Open  127.860001  149.899994  147.789993   

attribute        ABNB         ABT       ACGL         ACN        ADBE  ...  \
0          116.720001  108.533936  44.473526  284.401428  418.160004  ...   
1          116.720001  115.080002  46.770000  298.649994  418.160004  ...   
2          121.073997  118.000000  47.650002  306.709991  437.549988  ...   
3          114.394997  114.529999  46.139999  297.380005  413.790009  ...   
4          121.050003  117.849998  47.459999  304.130005  428.000000  ...   

attribute         WTW         WY       WYNN        XEL        XOM        XYL  \
0          200.021896  34.6353

In [32]:
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)
df

KeyError: 'high'